<img alt="Colaboratory logo" width="100%" src="https://raw.githubusercontent.com/LuisCSGuimaraes/FraudesEmCartoesCreditoML/main/Images/Banner.png">


*Um projeto inspirado por:*

<img alt="Colaboratory logo" width="10%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

**Data Science na Prática 2.0**
*by [sigmoidal.ai](https://sigmoidal.ai)*


# Detecção de fraudes em cartões de crédito utilizando Machine Learning

É importante que empresas de cartões de créditos sejam capazes de identificar transações fraudulentas de modo que seus clientes não sejam cobrados por compras que não realizaram.
Segundo o site Procon Online, o Brasil ocupa a segunda colocação entre os países que mais sofrem fraudes com cartões de crédito, débito e pré-pago, atrás apenas do México ([Fonte](https://www.procononline.com.br/fraude-com-cartao-de-credito/)).

<p align=center>
<img src="https://raw.githubusercontent.com/LuisCSGuimaraes/FraudesEmCartoesCreditoML/main/Images/FraudeCartaoCredito.jpg" width="60%"></p>

O primeiro semestre de 2021 registrou uma alta de quase 33% nas tentativas de fraudes em cartões de crédito ([Fonte](https://www.infomoney.com.br/minhas-financas/brasil-teve-alta-de-quase-33-nas-tentativas-de-fraude-com-cartao-de-credito-no-1-semestre-mostra-estudo/)). Por todos esses motivos, o investimento na área de detecção de fraudes por meio de Inteligência Artificial vem crescendo a cada ano, representando uma grande oportunidade em *Data Science*. 

Dispondo de grandes volumes de dados como base histórica, um algoritmo de *machine learning* apenas um pouco melhor que os anteriores já representa uma economia de milhões de Reais. Esse projeto busca encontrar um bom modelo de *machine learning* para realizar a detecção de fraudes em transações feitas por cartões de crédito.

# Índice
* [Fonte e obtenção dos dados](#fonte_dados)
* [Análise exploratória dos dados](#analise_dados)

## Fonte dos dados <a name='fonte_dados'></a>

Os dados utilizados neste projeto foram retirados do [kaggle](https://www.kaggle.com/), que é uma é uma comunidade on-line, subsidiária da Google, de cientistas de dados e profissionais de machine learning. O *dataset* contém transações realizadas por europeus em setembro de 2013 durante dois dias, período no qual foram registradas 284.807 transações, contendo um total de 492 fraldes. Por esse motivo este é um conjunto de dados extremamente desbalanceado, onde as fraudes representam apenas 0,17% do total.

Outro detalhe interessante é que as *features* são todas numéricas, e foram descaracterizadas (devido à privacidade e a segurança). Assim, os nomes das colunas são representados por $[V1, V2, V3 \dots, V28]$ 

[Na página original dos dados](https://www.kaggle.com/mlg-ulb/creditcardfraud), também é informado que as variáveis passaram por uma transformação conhecida como Análise de Componentes Principais (*Principal Component Analysis* - PCA), que permite a redução da dimensionalidade enquanto mantém o maior número possível de informações. Para conseguir isso, o algoritmo encontra um conjunto novo de recursos - os chamados **componentes**. Esses componentes são em número menor or igual às variáveis originais. No caso deste projeto, os componentes achados pela transformação da PCA são as próprias colunas $[V1, V2, V3 \dots, V28]$.

O *dataset* [original](https://www.kaggle.com/mlg-ulb/creditcardfraud) foi copiado para o meu dropbox e pode ser encontrado através deste [link](https://www.dropbox.com/s/jcnd83bfbdjm45b/creditcard.csv?dl=0)

In [3]:
# Importando os pacotes para análise de dados
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#Importando os pacotes para o modelo de Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

Para facilitar a aquisição dos dados, criei um *link* com acesso direto ao `csv`. Com isso, basta importar o conjunto de dados usando o método `pd.read_csv()`.

In [8]:
# Importando os dados
df = pd.read_csv("https://www.dropbox.com/s/jcnd83bfbdjm45b/creditcard.csv?dl=1")


In [21]:
print("####-Estrutura do dataframe-####\n")
print("Linhas: ", df.shape[0])
print("Colunas: ", df.shape[1])
print("--------------------------------")

df.head()

####-Estrutura do dataframe-####

Linhas:  284807
Colunas:  31
--------------------------------


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


Com os dados importados para dentro de uma estrutura *Dataframe* - e não havendo a necessidade de mais nenhum ajuste ou configuração nesta etapa, pode-se iniciar uma análise exploratória dos dados a fim de preparar um modelo de *Machine Learning*.

## Análise Exploratória <a name='analise_dados'></a>

Abordar tópicos/informações como:

* Ver as 5 primeiras entradas
* Ver o resumo estatístico do dataframe
* Verificar se há valores ausentes
* Plotar um gráfico de barras (ou countplot) para ver o balanceamento do *dataset*
* Plotar os seguintes histogramas:
    * Variável `Time`
        * Fraude (`Class == 1`)
        * Normal (`Class == 0`)
    * Variável `Amount`
        * Fraude (`Class == 1`)
        * Normal (`Class == 0`)
* Plotar um `boxplot` para a variável `Amount` quando houve fraude (`Class == 1`)
* Plotar uma matriz de correlação

## Preparação dos Dados

* Normalizar os dados que ainda não haviam sido pré-processados (`Time` e `Amount`)
* Dividir o conjunto de dados entre treino e teste
* [*Recomendado*] Balancear o conjunto de dados

## Modelo de Machine Learning

* Construir um modelo para **classificação**.
* [*Opcional*] Construir mais de um modelo para avaliar os desempenhos

## Avaliar o desempenho do modelo

* Identificar a melhor métrica para esse tipo de modelo
* [*Opcional*] Comparar o desempenho entre diversos modelos

## Conclusão

* Escrever suas conclusões a respeito da construção do modelo